In [1]:
import os
import pandas as pd
from glob import glob
from sklearn.model_selection import StratifiedKFold

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [2]:
dataset_root = "Skin cancer ISIC The International Skin Imaging Collaboration/Train"

image_paths = []
labels = []

class_names = sorted(os.listdir(dataset_root))  # keep consistent class order

for label in class_names:
    class_dir = os.path.join(dataset_root, label)
    if os.path.isdir(class_dir):
        img_files = glob(os.path.join(class_dir, "*.jpg")) + \
                    glob(os.path.join(class_dir, "*.png")) + \
                    glob(os.path.join(class_dir, "*.jpeg"))
        image_paths.extend(img_files)
        labels.extend([label] * len(img_files))

df = pd.DataFrame({"image": image_paths, "label": labels})
print(df.head())
print("Total images:", len(df))
print("Classes:", class_names)


                                               image              label
0  Skin cancer ISIC The International Skin Imagin...  actinic keratosis
1  Skin cancer ISIC The International Skin Imagin...  actinic keratosis
2  Skin cancer ISIC The International Skin Imagin...  actinic keratosis
3  Skin cancer ISIC The International Skin Imagin...  actinic keratosis
4  Skin cancer ISIC The International Skin Imagin...  actinic keratosis
Total images: 2239
Classes: ['actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus', 'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']


In [3]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["label_idx"] = le.fit_transform(df["label"])

num_classes = len(le.classes_)
print("Class to Index Mapping:", dict(zip(le.classes_, range(num_classes))))


Class to Index Mapping: {'actinic keratosis': 0, 'basal cell carcinoma': 1, 'dermatofibroma': 2, 'melanoma': 3, 'nevus': 4, 'pigmented benign keratosis': 5, 'seborrheic keratosis': 6, 'squamous cell carcinoma': 7, 'vascular lesion': 8}


In [4]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

df["fold"] = -1
for fold, (_, val_idx) in enumerate(skf.split(df["image"], df["label_idx"])):
    df.loc[val_idx, "fold"] = fold

df["fold"].value_counts()


fold
3    448
0    448
1    448
2    448
4    447
Name: count, dtype: int64

In [5]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.2)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05),
    transforms.GaussianBlur(kernel_size=3),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [6]:
class SkinCancerDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, "image"]
        label = self.df.loc[idx, "label_idx"]

        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        return img, label


In [7]:
def get_dataloaders(fold, batch_size=32):
    train_df = df[df.fold != fold]
    val_df = df[df.fold == fold]

    train_dataset = SkinCancerDataset(train_df, transform=train_transform)
    val_dataset = SkinCancerDataset(val_df, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size,
                            shuffle=False, num_workers=2)
    return train_loader, val_loader


In [8]:
import torch.nn as nn
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

def create_model(num_classes):
    weights = EfficientNet_B0_Weights.DEFAULT  # pretrained
    
    model = efficientnet_b0(weights=weights)

    # Freeze feature extractor initially
    for param in model.features.parameters():
        param.requires_grad = False

    # Replace classifier for 9 classes
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, num_classes)

    return model.to(device)


In [9]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(df['label_idx']),
    y=df['label_idx']
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
criterion


CrossEntropyLoss()

In [10]:
def get_optimizer(model, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    return optimizer


In [11]:
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

def train_one_epoch(model, loader, optimizer):
    model.train()
    total_loss = 0
    preds, targets = [], []
    
    for imgs, labels in tqdm(loader, desc="Training"):
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        preds.extend(outputs.argmax(dim=1).cpu().numpy())
        targets.extend(labels.cpu().numpy())

    acc = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds, average='macro')
    return total_loss/len(loader), acc, f1


def validate_one_epoch(model, loader):
    model.eval()
    total_loss = 0
    preds, targets = [], []
    
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Validating"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds.extend(outputs.argmax(dim=1).cpu().numpy())
            targets.extend(labels.cpu().numpy())

    acc = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds, average='macro')
    return total_loss/len(loader), acc, f1


In [12]:
EPOCHS = 10  # can increase later if time allows
BATCH_SIZE = 32

fold_performance = []

for fold in range(5):
    print(f"\n===== FOLD {fold+1} / 5 =====")
    
    train_loader, val_loader = get_dataloaders(fold, batch_size=BATCH_SIZE)
    model = create_model(num_classes)
    optimizer = get_optimizer(model)

    best_f1 = 0
    best_model_path = f"best_model_fold{fold}.pt"

    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        
        train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, optimizer)
        val_loss, val_acc, val_f1 = validate_one_epoch(model, val_loader)

        print(f"Train Loss: {train_loss:.4f} | ACC: {train_acc:.4f} | F1: {train_f1:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | ACC: {val_acc:.4f} | F1: {val_f1:.4f}")

        # Save best fold model
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), best_model_path)
            print(">>> Saved best model so far!")

    fold_performance.append({
        "fold": fold,
        "best_f1": best_f1
    })

print("\n=== Final Fold Results ===")
print(pd.DataFrame(fold_performance))


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /home/rishabhjp/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth



===== FOLD 1 / 5 =====


100%|██████████████████████████████████████| 20.5M/20.5M [00:01<00:00, 16.4MB/s]



Epoch 1/10


Validating: 100%|███████████████████████████████| 14/14 [00:04<00:00,  3.46it/s]


...

Train Loss: 1.4141 | ACC: 0.4643 | F1: 0.4402
Val   Loss: 1.4896 | ACC: 0.4474 | F1: 0.4233

Epoch 9/10


Validating: 100%|███████████████████████████████| 14/14 [00:03<00:00,  4.56it/s]


Train Loss: 1.4261 | ACC: 0.4682 | F1: 0.4462
Val   Loss: 1.4902 | ACC: 0.4318 | F1: 0.4161

Epoch 10/10


Validating: 100%|███████████████████████████████| 14/14 [00:03<00:00,  4.52it/s]

Train Loss: 1.3738 | ACC: 0.4838 | F1: 0.4624
Val   Loss: 1.4509 | ACC: 0.4586 | F1: 0.4437

=== Final Fold Results ===
   fold   best_f1
0     0  0.496377
1     1  0.461755
2     2  0.429575
3     3  0.474940
4     4  0.449723


In [14]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd

# Paths
BEST_MODEL_PATH = "best_model_fold0.pt"  # <- Update fold if needed
TEST_DIR = "Skin cancer ISIC The International Skin Imaging Collaboration/Test"

# GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Same transforms used in validation
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load test dataset
test_dataset = datasets.ImageFolder(TEST_DIR, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load model (EfficientNet B0)
from torchvision.models import efficientnet_b0
model = efficientnet_b0(pretrained=False)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, len(test_dataset.classes))
model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device))
model.to(device)
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = torch.argmax(F.softmax(outputs, dim=1), dim=1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Metrics
print("\n=== TEST SET RESULTS ===")
print(classification_report(all_labels, all_preds, target_names=test_dataset.classes, digits=4))

cm = confusion_matrix(all_labels, all_preds)
cm_df = pd.DataFrame(cm, index=test_dataset.classes, columns=test_dataset.classes)

print("\n=== CONFUSION MATRIX ===")
print(cm_df)


/home/rishabhjp/anaconda3/envs/miniproject/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rishabhjp/anaconda3/envs/miniproject/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_9506/2752352276.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for 


=== TEST SET RESULTS ===
                            precision    recall  f1-score   support

         actinic keratosis     0.8000    0.5000    0.6154        16
      basal cell carcinoma     0.5455    0.3750    0.4444        16
            dermatofibroma     0.6667    0.5000    0.5714        16
                  melanoma     0.2000    0.0625    0.0952        16
                     nevus     0.5000    0.9375    0.6522        16
pigmented benign keratosis     0.3846    0.6250    0.4762        16
      seborrheic keratosis     0.2500    0.3333    0.2857         3
   squamous cell carcinoma     0.3333    0.2500    0.2857        16
           vascular lesion     0.3750    1.0000    0.5455         3

                  accuracy                         0.4746       118
                 macro avg     0.4506    0.5093    0.4413       118
              weighted avg     0.4810    0.4746    0.4470       118


=== CONFUSION MATRIX ===
                            actinic keratosis  basal cell car